# Import libraries

In [ ]:
import xarray as xr
from glob import glob
import numpy as np
pattern_netcdf = 'F:/data/sat_data/aviso/data/*/*.nc'
pattern_zarr = 'F:/data/sat_data/aviso/zarr/'

In [ ]:
%%time
ds_zarr= xr.open_zarr(pattern_zarr)
ds_zarr

# Reading netCDF files

In [ ]:
%%time
#list files
files = [x for x in glob(pattern_netcdf)]
len(files)

In [ ]:
#only get new ones
files[-121]

In [ ]:
ds=xr.open_mfdataset(files[-121:],combine='nested',concat_dim='time')
ds.close()
#remove any duplicates
_, index = np.unique(ds['time'], return_index=True)
ds=ds.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
ds_netcdf = ds.chunk({'time':1000,'latitude':180,'longitude':180})

In [ ]:
# concat old & new data
ds_new = xr.concat([ds_zarr,ds_netcdf],dim='time')
#remove any duplicates
_, index = np.unique(ds_new['time'], return_index=True)
ds_new=ds_new.isel(time=index)


In [ ]:
ds_new

In [ ]:
#write zarr append
ds_netcdf.to_zarr(pattern_zarr, mode='a', append_dim='time')

In [ ]:
ds_zarr= xr.open_zarr(pattern_zarr)
ds_zarr